In [ ]:
import catboost as cb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv('first_round_training_data.csv')
df_test = pd.read_csv('first_round_testing_data.csv')
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
df_train['label'] = df_train['Quality_label'].map(dit)
# 将特征转换成数字
df_train=df_train.drop(labels=['Attribute1','Attribute2','Attribute3','Attribute4','Attribute5','Attribute6','Attribute7',
                       'Attribute8','Attribute9','Attribute10','Quality_label', ],axis=1)

In [ ]:
data=df_train.drop(labels='label',axis=1)
target = df_train['label']
X_train,X_test,y_train,y_test =train_test_split(data,target,test_size=0.2)

In [ ]:
model = cb.CatBoostClassifier(iterations=30000,depth = 8,

                               learning_rate = 0.0009,

                               custom_loss='AUC',

                               eval_metric='AUC',

                               bagging_temperature=0.83,

                               od_type='Iter',

                               rsm = 0.78,

                               od_wait=2000,

                               metric_period = 400,

                               l2_leaf_reg = 5,

                               thread_count = 20,

                               random_seed = 967,loss_function = 'MultiClass'

                              )

model.fit(X_train, y_train, eval_set=(X_test, y_test),use_best_model=True)

In [ ]:
preds_class = model.predict(df_test.drop(labels='Group',axis=1))

In [ ]:
example = pd.read_csv('datalab/37693/submit_example.csv')
result = pd.DataFrame({'Group':example.Group})

In [ ]:
df_test['label'] =preds_class

In [ ]:
for group_num in range(120):
#     print(group_num)
    all_counts = df_test.loc[df_test['Group'] ==group_num]['label'].value_counts()
#     print(all_counts)
    try:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = all_counts[0]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Good ratio'] = all_counts[1]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Good ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = all_counts[2]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = all_counts[3]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = 0

In [ ]:
result.to_csv(r'/home/tianchi/myspace/result.csv',index=False)